In [1]:
import os
import glob
import shutil
import numpy as np
from PIL import Image

In [2]:
def calc_perimeters (coords) :
    '''
    Args:
        coords: list of x-y coordinates of 4 coners [x1,y1,x2,y2,x3,y3,x4,y4]
    '''
    order = [0,2,4,6,0]
    perimeters = 0
    for i in range(4) :
        o = np.array(coords[order[i]:order[i]+2])
        d = np.array(coords[order[i+1]:order[i+1]+2])
        perimeters += np.linalg.norm(o-d) 
    return perimeters

In [4]:
DATASETS = ['train', 'val']

In [5]:
for DATASET in DATASETS :
    ORG_PATH = './{}'.format(DATASET) # make your own path
    DEST_PATH = './../Vehicles_Tmax250_/{}/'.format(DATASET) # make your own path

    txt_files = glob.glob(os.path.join(ORG_PATH, 'labelTxt-v2.0', '*.txt'))
    img_files = glob.glob(os.path.join(ORG_PATH, 'images','*.png'))
    meta_files = glob.glob(os.path.join(ORG_PATH, 'meta', '*.txt'))

    DEST_PATH_IMG = os.path.join(DEST_PATH, 'images')
    DEST_PATH_LBL = os.path.join(DEST_PATH, 'labelTxt')
    DEST_PATH_META = os.path.join(DEST_PATH, 'meta')

    dest_paths = [DEST_PATH_IMG, DEST_PATH_LBL, DEST_PATH_META]

    for dest_path in dest_paths :
        if os.path.exists(dest_path) :
            shutil.rmtree(dest_path, ignore_errors=True)
        os.makedirs(dest_path)


    for txt_file in txt_files :
        file_name = os.path.split(txt_file)[1] # 'P0001.txt'    
        img_name = file_name.replace('txt', 'png')
        org_img_file = os.path.join(ORG_PATH, 'images', img_name)
        org_meta_file = os.path.join(ORG_PATH, 'meta', file_name)

        with open(txt_file, 'r') as f1 :
            is_vehicle = False
            perimeters = []
            lines = f1.readlines()
            with open (os.path.join(DEST_PATH, 'tmp.txt'), 'w') as f2 :
                for line in lines :
                    if 'large-vehicle' in line :
                        is_vehicle = True
                        label = line.split(' ')
                        coords = [float(x) for x in label[:8]]
                        perimeter = calc_perimeters(coords)
                        perimeters.append(perimeter) 

                        f2.write(line)
                    elif 'small-vehicle' in line :
                        f2.write(line)
            if is_vehicle :
                if max(perimeters) > 250 :
                    dest_img_file = os.path.join(DEST_PATH_IMG, img_name)
                    dest_meta_file = os.path.join(DEST_PATH_META, file_name)
                    dest_lbl_file = os.path.join(DEST_PATH_LBL, file_name)

                    shutil.copy(org_img_file, dest_img_file)
                    shutil.copy(org_meta_file, dest_meta_file)
                    shutil.copy(os.path.join(DEST_PATH, 'tmp.txt'), dest_lbl_file)

            os.remove(os.path.join(DEST_PATH, 'tmp.txt'))  

# remove too large images

In [21]:
for DATASET in DATASETS :
    ORG_PATH = './{}'.format(DATASET) # make your own path
    DEST_PATH = './../Vehicles_max250_under3000/{}/'.format(DATASET) # make your own path

    txt_files = glob.glob(os.path.join(ORG_PATH, 'labelTxt-v2.0', '*.txt'))
    img_files = glob.glob(os.path.join(ORG_PATH, 'images','*.png'))
    meta_files = glob.glob(os.path.join(ORG_PATH, 'meta', '*.txt'))

    DEST_PATH_IMG = os.path.join(DEST_PATH, 'images')
    DEST_PATH_LBL = os.path.join(DEST_PATH, 'labelTxt')
    DEST_PATH_META = os.path.join(DEST_PATH, 'meta')

    dest_paths = [DEST_PATH_IMG, DEST_PATH_LBL, DEST_PATH_META]

    for dest_path in dest_paths :
        if os.path.exists(dest_path) :
            shutil.rmtree(dest_path, ignore_errors=True)
        os.makedirs(dest_path)


    for txt_file in txt_files :
        file_name = os.path.split(txt_file)[1] # 'P0001.txt'    
        img_name = file_name.replace('txt', 'png')
        org_img_file = os.path.join(ORG_PATH, 'images', img_name)
        org_meta_file = os.path.join(ORG_PATH, 'meta', file_name)
        
        with open(txt_file, 'r') as f1 :
            is_vehicle = False
            perimeters = []
            lines = f1.readlines()
            with open (os.path.join(DEST_PATH, 'tmp.txt'), 'w') as f2 :
                for line in lines :
                    if 'large-vehicle' in line :
                        is_vehicle = True
                        label = line.split(' ')
                        coords = [float(x) for x in label[:8]]
                        perimeter = calc_perimeters(coords)
                        perimeters.append(perimeter) 

                        f2.write(line)
                    elif 'small-vehicle' in line :
                        f2.write(line)
            if is_vehicle :
                if max(perimeters) > 250 :
                    w,h = Image.open(org_img_file).size
                    if min (w,h) <= 3000 :
                        dest_img_file = os.path.join(DEST_PATH_IMG, img_name)
                        dest_meta_file = os.path.join(DEST_PATH_META, file_name)
                        dest_lbl_file = os.path.join(DEST_PATH_LBL, file_name)

                        shutil.copy(org_img_file, dest_img_file)
                        shutil.copy(org_meta_file, dest_meta_file)
                        shutil.copy(os.path.join(DEST_PATH, 'tmp.txt'), dest_lbl_file)

            os.remove(os.path.join(DEST_PATH, 'tmp.txt'))  

In [24]:
DATASETS = ['train', 'val']
for DATASET in DATASETS :
    DEST_PATH = './../Vehicles_max250_under3000/{}/'.format(DATASET)
    dest_txt_files = glob.glob(os.path.join(DEST_PATH, 'labelTxt', '*.txt'))

    large = 0
    small = 0

    for dest_txt_file in dest_txt_files :
        file_name = os.path.split(dest_txt_file)[1]
        img_name = file_name.replace('txt', 'png')
        img_path = os.path.join(DEST_PATH, 'images', img_name)
        w,h = Image.open(img_path).size
        
        with (open(dest_txt_file, 'r')) as f :
            lines = f.readlines()
            for line in lines :
                if 'large' in line :
                    large +=1
                elif 'small' in line :
                    small +=1
    print ('dataset:{:5}, num_of_files:{:3}, num_of_large: {:3}, num_of_small: {:3}'.format(
        DATASET, len(dest_txt_files), large, small))

dataset:train, num_of_files:120, num_of_large: 6430, num_of_small: 8120
dataset:val  , num_of_files: 38, num_of_large: 2078, num_of_small: 2524


# Count instances by label

In [31]:
DATASETS = ['train', 'val']
for DATASET in DATASETS :
    DEST_PATH = './../Trucks/{}/'.format(DATASET)
    dest_txt_files = glob.glob(os.path.join(DEST_PATH, 'labelTxt', '*.txt'))

    large = 0

    for dest_txt_file in dest_txt_files :
        with (open(dest_txt_file, 'r')) as f :
            lines = f.readlines()
            for line in lines :
                large +=1
    print ('dataset:{:5}, num_of_files:{:3}, num_of_instances: {:3}'.format(DATASET, len(dest_txt_files), large))

dataset:train, num_of_files:131, num_of_instances: 4113
dataset:val  , num_of_files: 35, num_of_instances: 1194


In [38]:
# 250
DATASETS = ['train', 'val']
for DATASET in DATASETS :
    DEST_PATH = './../Trucks_250/{}/'.format(DATASET)
    dest_txt_files = glob.glob(os.path.join(DEST_PATH, 'labelTxt', '*.txt'))

    large = 0

    for dest_txt_file in dest_txt_files :
        with (open(dest_txt_file, 'r')) as f :
            lines = f.readlines()
            for line in lines :
                large +=1
    print ('dataset:{:5}, num_of_files:{:3}, num_of_instances: {:3}'.format(DATASET, len(dest_txt_files), large))

dataset:train, num_of_files: 25, num_of_instances: 209
dataset:val  , num_of_files:  6, num_of_instances:  33


In [42]:
# 200
DATASETS = ['train', 'val']
for DATASET in DATASETS :
    DEST_PATH = './../Trucks_200/{}/'.format(DATASET)
    dest_txt_files = glob.glob(os.path.join(DEST_PATH, 'labelTxt', '*.txt'))

    large = 0

    for dest_txt_file in dest_txt_files :
        with (open(dest_txt_file, 'r')) as f :
            lines = f.readlines()
            for line in lines :
                large +=1
    print ('dataset:{:5}, num_of_files:{:3}, num_of_instances: {:3}'.format(DATASET, len(dest_txt_files), large))

dataset:train, num_of_files: 52, num_of_instances: 1001
dataset:val  , num_of_files: 15, num_of_instances: 373


In [5]:
# 200
DATASETS = ['train', 'val']
for DATASET in DATASETS :
    DEST_PATH = './../Trucks_max200/{}/'.format(DATASET)
    dest_txt_files = glob.glob(os.path.join(DEST_PATH, 'labelTxt', '*.txt'))

    large = 0

    for dest_txt_file in dest_txt_files :
        with (open(dest_txt_file, 'r')) as f :
            lines = f.readlines()
            for line in lines :
                large +=1
    print ('dataset:{:5}, num_of_files:{:3}, num_of_instances: {:3}'.format(DATASET, len(dest_txt_files), large))

dataset:train, num_of_files:249, num_of_instances: 12399
dataset:val  , num_of_files: 69, num_of_instances: 3161


In [7]:
# 250
DATASETS = ['train', 'val']
for DATASET in DATASETS :
    DEST_PATH = './../Trucks_max250/{}/'.format(DATASET)
    dest_txt_files = glob.glob(os.path.join(DEST_PATH, 'labelTxt', '*.txt'))

    large = 0

    for dest_txt_file in dest_txt_files :
        with (open(dest_txt_file, 'r')) as f :
            lines = f.readlines()
            for line in lines :
                large +=1
    print ('dataset:{:5}, num_of_files:{:3}, num_of_instances: {:3}'.format(DATASET, len(dest_txt_files), large))

dataset:train, num_of_files:163, num_of_instances: 8572
dataset:val  , num_of_files: 45, num_of_instances: 2391


In [6]:
# 250 - Large and Small
DATASETS = ['train', 'val']
for DATASET in DATASETS :
    DEST_PATH = './../Vehicles_Tmax250_/{}/'.format(DATASET)
    dest_txt_files = glob.glob(os.path.join(DEST_PATH, 'labelTxt', '*.txt'))

    large = 0
    small = 0

    for dest_txt_file in dest_txt_files :
        with (open(dest_txt_file, 'r')) as f :
            lines = f.readlines()
            for line in lines :
                if 'large' in line :
                    large +=1
                elif 'small' in line :
                    small +=1
    print ('dataset:{:5}, num_of_files:{:3}, num_of_large: {:3}, num_of_small: {:3}'.format(
        DATASET, len(dest_txt_files), large, small))

dataset:train, num_of_files:163, num_of_large: 8572, num_of_small: 29021
dataset:val  , num_of_files: 45, num_of_large: 2391, num_of_small: 6707


In [13]:
from PIL import Image

In [16]:
# 250 - Large and Small
# high resolution


DATASETS = ['train', 'val']
for DATASET in DATASETS :
    DEST_PATH = './../Vehicles_Tmax250_/{}/'.format(DATASET)
    dest_txt_files = glob.glob(os.path.join(DEST_PATH, 'labelTxt', '*.txt'))

    large = 0
    small = 0

    for dest_txt_file in dest_txt_files :
        file_name = os.path.split(dest_txt_file)[1]
        img_name = file_name.replace('txt', 'png')
        img_path = os.path.join(DEST_PATH, 'images', img_name)
        w,h = Image.open(img_path).size
        if min (w,h) > 3000 :
            print (img_name, w, h)
            with (open(dest_txt_file, 'r')) as f :
                lines = f.readlines()
                for line in lines :
                    if 'large' in line :
                        large +=1
                    elif 'small' in line :
                        small +=1
    print ('dataset:{:5}, num_of_files:{:3}, num_of_large: {:3}, num_of_small: {:3}'.format(
        DATASET, len(dest_txt_files), large, small))

P0022.png 3946 6533
P0039.png 3599 3656
P0087.png 4439 5137
P1139.png 9089 6473
P1142.png 4018 4911
P1389.png 6008 5524
P1408.png 7253 5261
P1419.png 3373 4649
P1426.png 7905 5416
P1455.png 5780 5628
P1457.png 5193 4658
P1462.png 6138 4817
P1466.png 6327 5662
P1783.png 3147 3357
P1845.png 3918 3487
P2609.png 3923 4860
P2612.png 6725 4331
P2641.png 7574 5625
P2642.png 7039 5551
P3065.png 7360 4912
P3555.png 7360 4912
P4464.png 7360 4912
P4724.png 7360 4912
P4851.png 4096 4096
P5034.png 7360 4912
P5102.png 7360 4912
P5195.png 7360 4912
P5760.png 7360 4912
P6281.png 7360 4912
P6494.png 3433 4360
P6525.png 4097 4096
P7476.png 7360 4912
P7584.png 7360 4912
P8003.png 7360 4912
P8204.png 7360 4912
P8461.png 7360 4912
P8587.png 7360 4912
P9337.png 7360 4912
P9347.png 7360 4912
P9645.png 7360 4912
P10747.png 7360 4912
P10808.png 7360 4912
P10963.png 7360 4912
dataset:train, num_of_files:163, num_of_large: 2142, num_of_small: 20901
P1471.png 4801 5046
P1854.png 13383 4287
P4151.png 7360 4912
P50

In [10]:
dest_txt_file
file_name = os.path.split(dest_txt_file)[1]
img_name = file_name.replace('txt', 'png')
img_path = os.path.join(DEST_PATH, 'images', img_name)
w,h = Image.open(img_path).size
if min (w,h) > 3000 :
    print (w, h)

(1490, 2137)